# Random Forest

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import time
from sklearn.metrics import mean_absolute_error

In [2]:
# cargar el dataset
tiempo = pd.read_parquet('dataframe_final_1970-2023_alicante_alicante.parquet')

In [20]:
tiempo

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaPresMax_3,horaPresMax_-1,presMin_1,presMin_2,presMin_3,presMin_-1,horaPresMin_1,horaPresMin_2,horaPresMin_3,horaPresMin_-1
0,1970-01-01,8025,ALACANT/ALICANTE,ALICANTE,81,9.3,0.0,5.0,01:15:00,13.6,...,None,10:00:00,None,None,None,998.1,None,None,None,19:00:00
1,1970-01-02,8025,ALACANT/ALICANTE,ALICANTE,81,7.0,7.8,3.6,05:45:00,10.4,...,None,00:00:00,993.0,None,None,995.4,00:00:00,None,None,None
2,1970-01-03,8025,ALACANT/ALICANTE,ALICANTE,81,11.1,0.0,7.4,00:45:00,14.8,...,None,00:00:00,998.1,993.0,None,989.3,19:00:00,00:00:00,None,15:00:00
3,1970-01-04,8025,ALACANT/ALICANTE,ALICANTE,81,13.8,0.2,9.0,05:20:00,18.6,...,None,10:00:00,995.4,998.1,993.0,988.8,None,19:00:00,00:00:00,None
4,1970-01-05,8025,ALACANT/ALICANTE,ALICANTE,81,14.6,0.5,9.8,05:30:00,19.4,...,10:00:00,None,989.3,995.4,998.1,986.9,15:00:00,None,19:00:00,07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19353,2022-12-27,8025,ALACANT/ALICANTE,ALICANTE,81,13.3,0.0,6.6,07:00:00,20.0,...,None,00:00:00,1013.3,1010.4,1010.4,1013.8,00:00:00,05:00:00,14:00:00,None
19354,2022-12-28,8025,ALACANT/ALICANTE,ALICANTE,81,12.7,0.0,6.1,04:30:00,19.3,...,23:00:00,10:00:00,1018.5,1013.3,1010.4,1012.7,01:00:00,00:00:00,05:00:00,14:00:00
19355,2022-12-29,8025,ALACANT/ALICANTE,ALICANTE,81,14.2,0.0,6.6,02:10:00,21.9,...,23:00:00,23:00:00,1013.8,1018.5,1013.3,1013.9,None,01:00:00,00:00:00,None
19356,2022-12-30,8025,ALACANT/ALICANTE,ALICANTE,81,17.2,0.0,10.5,03:50:00,24.0,...,11:00:00,None,1012.7,1013.8,1018.5,1015.4,14:00:00,None,01:00:00,15:00:00


In [3]:
# 70% del tiempo para el train, 30% validacion (hasta 31 diciembre 2022)
# el conjunto de test cogeremos 2023

fecha_test = '2007-12-31'

In [4]:
# las variables del día siguiente las eliminamos y también las correspondientes a la localización ya que estamos solo en Alicante

model_columns = list(set(tiempo.columns) - set(['tmed_mañana', 'prec_mañana', 'tmin_mañana', 'horatmin_mañana', 'tmax_mañana',
                                                                                    'horatmax_mañana', 'dir_mañana', 'velmedia_mañana', 'racha_mañana', 'horaracha_mañana', 
                                                                                    'sol_mañana', 'presMax_mañana', 'horaPresMax_mañana', 'presMin_mañana', 'horaPresMin_mañana',
                                                                                    'indicativo', 'nombre', 'provincia', 'altitud', 'fecha']))

In [5]:
tiempo.horatmin = tiempo.horatmin.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax = tiempo.horatmax.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha = tiempo.horaracha.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaPresMax = tiempo.horaPresMax.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin = tiempo.horaPresMin.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horatmin_1 = tiempo.horatmin_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_2 = tiempo.horatmin_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_3 = tiempo.horatmin_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmin_mañana = tiempo.horatmin_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horatmax_1 = tiempo.horatmax_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_2 = tiempo.horatmax_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_3 = tiempo.horatmax_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horatmax_mañana = tiempo.horatmax_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaracha_1 = tiempo.horaracha_1.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_2 = tiempo.horaracha_2.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_3 = tiempo.horaracha_3.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)
tiempo.horaracha_mañana = tiempo.horaracha_mañana.apply(lambda x: pd.to_datetime(str(x), format='%H:%M:%S').round('H').hour if x is not None else np.nan)

tiempo.horaPresMax_1 = tiempo.horaPresMax_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_2 = tiempo.horaPresMax_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_3 = tiempo.horaPresMax_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMax_mañana = tiempo.horaPresMax_mañana.apply(lambda x: x.hour if x is not None else np.nan)

tiempo.horaPresMin_1 = tiempo.horaPresMin_1.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_2 = tiempo.horaPresMin_2.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_3 = tiempo.horaPresMin_3.apply(lambda x: x.hour if x is not None else np.nan)
tiempo.horaPresMin_mañana = tiempo.horaPresMin_mañana.apply(lambda x: x.hour if x is not None else np.nan)

In [6]:
tiempo['año'] = tiempo['fecha'].dt.year
tiempo['mes'] = tiempo['fecha'].dt.month
tiempo['dia'] = tiempo['fecha'].dt.day

In [7]:
train = tiempo[tiempo.fecha <= fecha_test].fillna(-1)
test = tiempo[tiempo.fecha > fecha_test].fillna(-1)

In [10]:
train

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,presMin_2,presMin_3,presMin_mañana,horaPresMin_1,horaPresMin_2,horaPresMin_3,horaPresMin_mañana,año,mes,dia
0,1970-01-01,8025,ALACANT/ALICANTE,ALICANTE,81,9.3,0.0,5.0,1.0,13.6,...,-1,-1,998.1,-1.0,-1.0,-1.0,19.0,1970,1,1
1,1970-01-02,8025,ALACANT/ALICANTE,ALICANTE,81,7.0,7.8,3.6,6.0,10.4,...,-1,-1,995.4,0.0,-1.0,-1.0,-1.0,1970,1,2
2,1970-01-03,8025,ALACANT/ALICANTE,ALICANTE,81,11.1,0.0,7.4,1.0,14.8,...,993.0,-1,989.3,19.0,0.0,-1.0,15.0,1970,1,3
3,1970-01-04,8025,ALACANT/ALICANTE,ALICANTE,81,13.8,0.2,9.0,5.0,18.6,...,998.1,993.0,988.8,-1.0,19.0,0.0,-1.0,1970,1,4
4,1970-01-05,8025,ALACANT/ALICANTE,ALICANTE,81,14.6,0.5,9.8,6.0,19.4,...,995.4,998.1,986.9,15.0,-1.0,19.0,7.0,1970,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13874,2007-12-27,8025,ALACANT/ALICANTE,ALICANTE,81,11.3,0.0,6.8,0.0,15.8,...,1013.9,1013.1,1019.5,5.0,17.0,2.0,16.0,2007,12,27
13875,2007-12-28,8025,ALACANT/ALICANTE,ALICANTE,81,9.9,0.0,4.0,6.0,15.8,...,1013.5,1013.9,1018.8,0.0,5.0,17.0,-1.0,2007,12,28
13876,2007-12-29,8025,ALACANT/ALICANTE,ALICANTE,81,9.4,0.0,2.5,8.0,16.2,...,1018.6,1013.5,1014.9,16.0,0.0,5.0,16.0,2007,12,29
13877,2007-12-30,8025,ALACANT/ALICANTE,ALICANTE,81,11.1,0.0,3.4,6.0,18.8,...,1019.5,1018.6,1014.6,-1.0,16.0,0.0,5.0,2007,12,30


In [11]:
train[model_columns]

,presMax_2,tmed_1,tmed_2,velmedia_3,racha_2,presMin,dir_1,sol_1,prec_2,racha,...,presMax_1,horaPresMin_1,presMax,sol_2,dir_2,sol_3,tmin_2,velmedia_1,horaracha_2,tmin_1
0,-1,-1,-1,-1,-1,993.0,-1.0,-1,-1,11.1,...,-1,-1.0,1000.6,-1,-1.0,-1,-1,-1,-1.0,-1
1,-1,9.3,-1,-1,-1,998.1,32.0,6.4,-1,4.7,...,1000.6,0.0,1001.0,-1,-1.0,-1,-1,3.1,-1.0,5.0
2,1000.6,7.0,9.3,-1,11.1,995.4,5.0,0.0,0.0,5.0,...,1001.0,19.0,998.2,6.4,32.0,-1,5.0,2.2,13.0,3.6
3,1001.0,11.1,7.0,3.1,4.7,989.3,23.0,3.5,7.8,6.4,...,998.2,-1.0,995.4,0.0,5.0,6.4,3.6,1.4,16.0,7.4
4,998.2,13.8,11.1,2.2,5.0,988.8,27.0,1.8,0.0,7.2,...,995.4,15.0,991.4,3.5,23.0,0.0,7.4,0.6,14.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13874,1016.7,12.1,11.8,1.9,6.1,1018.6,3.0,1.9,0.0,8.3,...,1018.6,5.0,1022.0,8.7,22.0,3.9,4.6,2.2,14.0,7.4
13875,1018.6,11.3,12.1,2.2,6.9,1019.5,10.0,7.5,9.5,5.8,...,1022.0,0.0,1021.5,1.9,3.0,8.7,7.4,2.2,15.0,6.8
13876,1022.0,9.9,11.3,2.2,8.3,1018.8,14.0,7.4,0.0,4.7,...,1021.5,16.0,1021.1,7.5,10.0,1.9,6.8,1.7,12.0,4.0
13877,1021.5,9.4,9.9,2.2,5.8,1014.9,15.0,8.7,0.0,6.1,...,1021.1,-1.0,1018.8,7.4,14.0,7.5,4.0,1.7,13.0,2.5


## Target: tmax_mañana

In [8]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500, 1000]:
    for max_depth in [5, 10, 20, 40, 60]:
        start_time = time.time()
        model = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = 'sqrt')
        model.fit(train[model_columns],train.tmax_mañana);

        train_pred = model.predict(train[model_columns])
        test_pred = model.predict(test[model_columns])

        metrics['RF_'+ str(n_estimators)+'_'+str(max_depth)] = {
            'Train_MAE': mean_absolute_error(train.tmax_mañana, train_pred),
            'Test_MAE': mean_absolute_error(test.tmax_mañana, test_pred),
            'Run_Time': time.time() - start_time
        }

metrics_RF = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_RF['delta'] = metrics_RF.Test_MAE - metrics_RF.Train_MAE
metrics_RF

,Run_Time,Train_MAE,Test_MAE,delta
RF_5_5,0.508226,1.639681,1.745070,0.105388
RF_5_10,0.573186,1.322397,1.748783,0.426386
RF_5_20,0.772933,0.759453,1.838214,1.078762
RF_5_40,0.728427,0.725394,1.838095,1.112701
RF_5_60,0.768964,0.722260,1.832407,1.110148
RF_10_5,0.548413,1.641715,1.747184,0.105469
RF_10_10,0.842769,1.287155,1.671534,0.384379
RF_10_20,1.170342,0.691148,1.734694,1.043546
RF_10_40,1.117017,0.659989,1.746965,1.086976
RF_10_60,1.132355,0.660511,1.735404,1.074893


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos °C de media se equivoca más en el Test.

In [9]:
metrics_RF.to_parquet('rf_tmax.parquet')

## Target: tmin_mañana

In [10]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500, 1000]:
    for max_depth in [5, 10, 20, 40, 60]:
        start_time = time.time()
        model = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = 'sqrt')
        model.fit(train[model_columns],train.tmin_mañana);

        train_pred = model.predict(train[model_columns])
        test_pred = model.predict(test[model_columns])

        metrics['RF_'+ str(n_estimators)+'_'+str(max_depth)] = {
            'Train_MAE': mean_absolute_error(train.tmin_mañana, train_pred),
            'Test_MAE': mean_absolute_error(test.tmin_mañana, test_pred),
            'Run_Time': time.time() - start_time
        }

metrics_RF = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_RF['delta'] = metrics_RF.Test_MAE - metrics_RF.Train_MAE
metrics_RF

,Run_Time,Train_MAE,Test_MAE,delta
RF_5_5,0.604386,1.555854,1.624878,0.069024
RF_5_10,0.626323,1.171182,1.441915,0.270733
RF_5_20,1.123903,0.642752,1.479918,0.837166
RF_5_40,0.839266,0.632579,1.546618,0.914039
RF_5_60,0.843132,0.630287,1.496317,0.866029
RF_10_5,0.815332,1.470581,1.488561,0.017981
RF_10_10,0.781428,1.142499,1.403086,0.260587
RF_10_20,1.222261,0.601942,1.411709,0.809768
RF_10_40,1.199463,0.565356,1.390042,0.824686
RF_10_60,1.229738,0.569851,1.389980,0.820129


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos °C de media se equivoca más en el Test.

In [11]:
metrics_RF.to_parquet('rf_tmin.parquet')

## Target: prec_mañana

In [12]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500, 1000]:
    for max_depth in [5, 10, 20, 40, 60]:
        start_time = time.time()
        model = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = 'sqrt')
        model.fit(train[model_columns],train.prec_mañana);

        train_pred = model.predict(train[model_columns])
        test_pred = model.predict(test[model_columns])

        metrics['RF_'+ str(n_estimators)+'_'+str(max_depth)] = {
            'Train_MAE': mean_absolute_error(train.prec_mañana, train_pred),
            'Test_MAE': mean_absolute_error(test.prec_mañana, test_pred),
            'Run_Time': time.time() - start_time
        }

metrics_RF = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_RF['delta'] = metrics_RF.Test_MAE - metrics_RF.Train_MAE
metrics_RF

,Run_Time,Train_MAE,Test_MAE,delta
RF_5_5,0.483709,1.433718,1.437467,0.003748
RF_5_10,0.472733,1.159259,1.642832,0.483573
RF_5_20,0.576458,0.722055,1.891735,1.169681
RF_5_40,0.615403,0.659382,1.826169,1.166787
RF_5_60,0.607376,0.632033,1.647034,1.015002
RF_10_5,0.485271,1.419530,1.440681,0.021151
RF_10_10,0.592230,1.155364,1.499507,0.344143
RF_10_20,0.905577,0.667890,1.628060,0.960169
RF_10_40,0.906576,0.638193,1.794801,1.156607
RF_10_60,0.920207,0.638142,1.788053,1.149911


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántos mm de media se equivoca más en el Test.

In [13]:
metrics_RF.to_parquet('rf_prec.parquet')

## Target: sol_mañana

In [14]:
metrics = {}
for n_estimators in [5, 10, 20, 50, 100, 200, 500, 1000]:
    for max_depth in [5, 10, 20, 40, 60]:
        start_time = time.time()
        model = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = 'sqrt')
        model.fit(train[model_columns],train.sol_mañana);

        train_pred = model.predict(train[model_columns])
        test_pred = model.predict(test[model_columns])

        metrics['RF_'+ str(n_estimators)+'_'+str(max_depth)] = {
            'Train_MAE': mean_absolute_error(train.sol_mañana, train_pred),
            'Test_MAE': mean_absolute_error(test.sol_mañana, test_pred),
            'Run_Time': time.time() - start_time
        }

metrics_RF = pd.DataFrame.from_dict(metrics, orient='index',columns=['Run_Time', 'Train_MAE', 'Test_MAE'])
metrics_RF['delta'] = metrics_RF.Test_MAE - metrics_RF.Train_MAE
metrics_RF

,Run_Time,Train_MAE,Test_MAE,delta
RF_5_5,0.418883,2.391255,2.550458,0.159203
RF_5_10,0.457798,1.954508,2.450862,0.496354
RF_5_20,0.613359,1.083319,2.532512,1.449192
RF_5_40,0.639743,0.992676,2.582858,1.590182
RF_5_60,0.704117,0.996101,2.547582,1.551481
RF_10_5,0.456588,2.358730,2.513175,0.154445
RF_10_10,0.626854,1.944545,2.428379,0.483835
RF_10_20,0.985882,1.012186,2.489958,1.477771
RF_10_40,0.988132,0.927398,2.508625,1.581226
RF_10_60,1.002831,0.921087,2.506227,1.585140


El delta es la diferencia de MAE's, el Train menos el Test. El resultado es cuántas h de media se equivoca más en el Test.

In [15]:
metrics_RF.to_parquet('rf_sol.parquet')